In [2]:
!pip install pandas
!pip install numpy
!pip install pandas-profiling==2.7.1
!pip install pycaret
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import os

!pip install seaborn
import seaborn as sns

from pandas.api.types import CategoricalDtype
#Train_label = pd.read_csv('Training set labels.csv')
#Train_value = pd.read_csv('Training set values.csv')
Train = pd.read_csv('Train_clean.csv')
#Train = pd.merge(Train_value,Train_label, how='inner',left_on='id',right_on='id', left_index=False)

  Using cached pandas_profiling-2.7.1-py2.py3-none-any.whl (252 kB)
  Using cached visions-0.4.1-py3-none-any.whl (58 kB)


  Attempting uninstall: visions
    Found existing installation: visions 0.7.1
    Uninstalling visions-0.7.1:
      Successfully uninstalled visions-0.7.1
  Attempting uninstall: pandas-profiling
    Found existing installation: pandas-profiling 3.0.0
    Uninstalling pandas-profiling-3.0.0:
      Successfully uninstalled pandas-profiling-3.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 2.3.2 requires pandas-profiling>=2.8.0, but you have pandas-profiling 2.7.1 which is incompatible.


  Using cached pandas_profiling-3.0.0-py2.py3-none-any.whl (248 kB)


  Using cached visions-0.7.1-py3-none-any.whl (102 kB)


  Attempting uninstall: visions
    Found existing installation: visions 0.4.1
    Uninstalling visions-0.4.1:
      Successfully uninstalled visions-0.4.1
  Attempting uninstall: pandas-profiling
    Found existing installation: pandas-profiling 2.7.1
    Uninstalling pandas-profiling-2.7.1:
      Successfully uninstalled pandas-profiling-2.7.1


In [2]:
from pycaret.utils import version
version()

'2.3.2'

In [3]:
Train.isnull().sum()

Unnamed: 0               0
id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
scheme_name              0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
s

In [4]:
Train['construction_year']

0        1999
1        2010
2        2009
3        1986
4        1960
         ... 
59395    1999
59396    1996
59397    1960
59398    1960
59399    2002
Name: construction_year, Length: 59400, dtype: int64

In [5]:
year_Cluster = Train[['longitude','latitude','region_code','installer']]
year_Cluster
#,'construction_year'
# ,'funder','installer'

,longitude,latitude,region_code,installer
0,34.938093,-9.856322,11,Roman
1,34.698766,-2.147466,20,GRUMETI
2,37.460664,-3.821329,21,World vision
3,38.486161,-11.155298,9,UNICEF
4,31.130847,-1.825359,8,Artisan
...,...,...,...,...
59395,37.169807,-3.253847,3,CES
59396,35.249991,-9.070629,11,Cefa
59397,34.017087,-8.750434,12,DWE
59398,35.861315,-6.378573,1,Musa


In [6]:
from pycaret.clustering import *
clu1 = setup(year_Cluster, session_id=123, log_experiment=True, log_plots = True, 
             experiment_name='construction')

,Description,Value
0,session_id,123
1,Original Data,"(59400, 4)"
2,Missing Values,False
3,Numeric Features,2
4,Categorical Features,2
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(59400, 2099)"
9,CPU Jobs,-1


In [7]:
models()

,Name,Reference
ID,,
kmeans,K-Means Clustering,sklearn.cluster._kmeans.KMeans
ap,Affinity Propagation,sklearn.cluster._affinity_propagation.Affinity...
meanshift,Mean Shift Clustering,sklearn.cluster._mean_shift.MeanShift
sc,Spectral Clustering,sklearn.cluster._spectral.SpectralClustering
hclust,Agglomerative Clustering,sklearn.cluster._agglomerative.AgglomerativeCl...
dbscan,Density-Based Spatial Clustering,sklearn.cluster._dbscan.DBSCAN
optics,OPTICS Clustering,sklearn.cluster._optics.OPTICS
birch,Birch Clustering,sklearn.cluster._birch.Birch
kmodes,K-Modes Clustering,kmodes.kmodes.KModes


In [ ]:
kmeans = create_model('kmeans', num_clusters = 10)

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,00:22:39
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Clusters
Estimator,. . . . . . . . . . . . . . . . . .,K-Means Clustering


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness


In [ ]:
kmodes = create_model('kmodes', num_clusters = 10)

In [ ]:
kmeans_results = assign_model(kmeans)
kmeans_results

In [ ]:
plot_model(kmeans)

In [ ]:
plot_model(kmeans, plot = 'distance')


In [ ]:
kmeans_results['construction_year'] = Train['construction_year']
kmeans_results

In [ ]:
kmeans_results.loc[kmeans_results['construction_year'] == 0]

In [ ]:
test_year = kmeans_results.loc[kmeans_results['construction_year'] != 0]
test_year

In [ ]:
pd.pivot_table(test_year,index=["region_code","Cluster"],values=["construction_year"],aggfunc=[np.mean]).astype(int)

In [ ]:
base_year = pd.DataFrame(pd.pivot_table(test_year,index=["Cluster"],values=["construction_year"],aggfunc=[np.mean]).astype(int).to_records())
base_year

In [ ]:
base_year = base_year.rename(columns={"('mean', 'construction_year')":'year'})
base_year

In [ ]:
dict = base_year.set_index('Cluster')['year'].to_dict()
dict['Cluster 0']
dict

In [ ]:
pd.pivot_table(test_year,index=["region_code"],values=["construction_year"],aggfunc=[np.mean]).astype(int)

In [ ]:
kmeans_results

In [ ]:
# mapp using base_year to replace zero 
#kmeans_results.loc[kmeans_results['construct_years'] == 0, 'construct_years' ] = base_year[:1]
#kmeans_results.loc[kmeans_results.Cluster.isin(base_year.Cluster),'construction_year'] = base_year['year']

#kmeans_results['construction_year'] = kmeans_results['construction_year'].apply(lambda x: x.Cluster.map(dict) if x == 0 else x) 
#df.customer.map(genders_dict)

#if kmeans_results['construction_year'].eq(0).any():
#    kmeans_results['construction_year'] = kmeans_results.Cluster.map(dict)

kmeans_results.loc[kmeans_results['construction_year'] == 0, 'construction_year'] = kmeans_results.Cluster.map(dict)
    
    

In [ ]:
kmeans_results


fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
sns.histplot(year_Cluster['construction_year'], ax=ax1)
sns.histplot(Construct_year['construction_year'],ax=ax2)
plt.show()


In [ ]:
kmeans_results.loc[kmeans_results['construction_year'] == 0]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
sns.histplot(Train['construction_year'], ax=ax1)
sns.histplot(kmeans_results['construction_year'],ax=ax2)
plt.show()

In [ ]:
Train['construction_year'] = kmeans_results['construction_year']

In [ ]:
Train['construction_year'].hist()

In [ ]:
Train.to_csv('Train_clean2.csv')

In [ ]:
Train.isnull().sum()